In [ ]:
def build_model():
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    import joblib
    df = pd.read_csv('logic_prediction.csv')
    X = df[['vibration', 'avg_speed', 'age']]
    y = df['did_break']
    model = LogisticRegression()
    model.fit(X, y)
    joblib.dump(model, "Logistic.joblib")
    
build_model()

In [ ]:
import joblib

# Chargement du modèle de regression et récupération des coefficient
model = joblib.load("Logistic.joblib")

coeff = [coef for coef in model.coef_]
intercept = model.intercept_

# On génère la chaîne de caractère contenant le code C et on l'ouvre
c_code = """
#include <stdio.h>

float exp_approx(float x, int n_term) {{
    float result = 1.0;
    float term = 1.0;
    int i;
    for (i = 1; i <= n_term; i++) {{
        term *= x / i;
        result += term;
    }}
    return result;
}}

float sigmoid(float x) {{
    float exponent = -x;
    float denominator = 1.0 + exp_approx(exponent, 10);
    return 1.0 / denominator;
}}

float logistic_regression(float* features, int n_parameter) {{
    float thetas[] = {{{', '.join(str(coeff) for coef in coeff[0].tolist())}}};
    float pred = {intercept[0]};
    for (int i = 0; i < n_parameter - 1; i++) {{
        z += features[i] * thetas[i+1];
    }}
    return sigmoid(z);
}}

int main() {{
    float features[] = {{1.5, 30.0, 120}}; // Exemple de données d'entrée
    int n_features = sizeof(features)/sizeof(float);
    float pred = logistic_regression(features, n_features);
    printf("Prediction : %f\\n", pred);
    return 0;
}}
"""

with open("modeleLogistic.c", "w") as f:
    f.write(c_code)

# Compilation du code C
compilation_command = "gcc modeleLogistic.c -o modeleLogistic"
print("Commande de compilation : {}".format(compilation_command))

Commande de compilation : gcc modeleLogistic.c -o modeleLogistic
